In [13]:
# 라이브러리 로드
import os
import json
import csv
import requests
import folium
import pandas as pd
import numpy as np
import platform
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
import re

# total_Seoul = pd.read_excel('행정구역_서울_주민등록세대수_2024.xlsx')
# total_Seoul.head(10)

In [6]:
# 현재 작업 디렉토리 경로 얻기
current_directory = os.getcwd()
# 나눔바른고딕 글꼴 파일 경로 지정
font_path = os.path.join(current_directory, "NanumBarunGothic.ttf")

# 글꼴이 있는지 확인
if os.path.exists(font_path):
    # 폰트 추가
    fm.fontManager.addfont(font_path)

    # 캐시를 새로 고치지 않고 직접 폰트 설정
    font_prop = fm.FontProperties(fname=font_path)
    plt.rc("font", family=font_prop.get_name())
    plt.rcParams["axes.unicode_minus"] = False  # 마이너스 부호 출력 설정

In [14]:
df = pd.read_csv('catch_loc_updated_0630.csv')
df

,Unnamed: 0,Name,Category,Category_1,Location,Location_1,Label,Rating,Review,avg_lunch,avg_dinner,min_lunch,max_lunch,min_dinner,max_dinner,Location_2
0,0,런던 베이글 뮤지엄 도산,"카페,디저트",카페,도산공원,강남구,웨이팅,4.4,3419,15000.0,15000.0,10000.0,20000.0,10000.0,20000.0,도산공원
1,1,산청숯불가든 마곡,돼지고기구이,한식,마곡,강서구,웨이팅,4.6,583,40000.0,40000.0,20000.0,60000.0,20000.0,60000.0,마곡
2,2,라프레플루트,"카페,디저트",카페,서울숲,성동구,웨이팅,4.3,278,20000.0,20000.0,10000.0,30000.0,10000.0,30000.0,서울숲
3,3,한소반 청계산점,"육류,고기요리",한식,청계산,서초구,웨이팅,4.3,197,15000.0,15000.0,10000.0,20000.0,10000.0,20000.0,청계산
4,4,크레이지카츠,돈가스,한식,합정,마포구,웨이팅,4.7,562,15000.0,15000.0,10000.0,20000.0,10000.0,20000.0,합정
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3397,3397,돈구멍 마곡본점,돼지고기구이,한식,마곡,강서구,예약-웨이팅,5.0,41,35000.0,35000.0,30000.0,40000.0,30000.0,40000.0,마곡
3398,3398,하이디라오 서초점,중식,중식,강남,강남구,예약-웨이팅,4.7,894,40000.0,40000.0,30000.0,50000.0,30000.0,50000.0,강남
3399,3399,하이디라오 영등포점,중식,중식,영등포,영등포구,예약-웨이팅,4.7,950,40000.0,40000.0,30000.0,50000.0,30000.0,50000.0,영등포
3400,3400,라이언하트,"칵테일,위스키",주류,송파,송파구,예약-웨이팅,4.8,111,영업안함,50000.0,NaN,NaN,40000.0,60000.0,송파


In [4]:
df.to_excel('a.xlsx')

In [15]:
print(df.Category_1.value_counts())

Category_1
한식       946
양식       928
일식       669
주류       414
카페       123
중식       123
퓨전        92
아시아음식     51
육류        38
뷔페        18
Name: count, dtype: int64


In [16]:
s_data = df.copy()

# 가장 많은 음식점 수인 4지역 필터일
top_locations = s_data['Location_1'].value_counts().nlargest(4).index
# print(top_locations)

# 선택된 지역구들에 해당하는 데이터 필터링
filtered_data = s_data[s_data['Location_1'].isin(top_locations)]
print(filtered_data)

# 전처리
# avg_lunch와 avg_dinner 컬럼에서 NA 값을 0으로 대체
filtered_data['avg_lunch'] = filtered_data['avg_lunch'].fillna(0)
filtered_data['avg_dinner'] = filtered_data['avg_dinner'].fillna(0)

# avg_lunch와 avg_dinner 컬럼에서 '영업안함'과 '가격변동' 값을 0으로 대체
filtered_data['avg_lunch'] = filtered_data['avg_lunch'].replace(['영업안함', '가격변동'], 0)
filtered_data['avg_dinner'] = filtered_data['avg_dinner'].replace(['영업안함', '가격변동'], 0)

# avg_lunch와 avg_dinner 컬럼을 float 타입으로 변환
filtered_data['avg_lunch'] = filtered_data['avg_lunch'].astype(float)
filtered_data['avg_dinner'] = filtered_data['avg_dinner'].astype(float)

filtered_data

      Unnamed: 0           Name Category Category_1 Location Location_1  \
0              0  런던 베이글 뮤지엄 도산   카페,디저트         카페     도산공원        강남구   
2              2         라프레플루트   카페,디저트         카페      서울숲        성동구   
4              4         크레이지카츠      돈가스         한식       합정        마포구   
9              9             난포       한식         한식      서울숲        성동구   
10            10         미뉴트빠삐용   카페,디저트         카페   압구정로데오        강남구   
...          ...            ...      ...        ...      ...        ...   
3393        3393       온천집 스키야키     샤브샤브         한식   압구정로데오        강남구   
3394        3394          함루 본점       일식         일식       공덕        마포구   
3395        3395        이치류 한남점      양고기         육류       한남        용산구   
3398        3398      하이디라오 서초점       중식         중식       강남        강남구   
3401        3401       백송한우 청담점    소고기구이         한식       청담        강남구   

       Label  Rating  Review avg_lunch avg_dinner  min_lunch  max_lunch  \
0        웨이팅     4.4    

C:\Users\Administrator\AppData\Local\Temp\ipykernel_6740\3224175016.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['avg_lunch'] = filtered_data['avg_lunch'].fillna(0)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_6740\3224175016.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['avg_dinner'] = filtered_data['avg_dinner'].fillna(0)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_6740\3224175016.py:17: SettingWithCopyWarning: 
A value is trying to be set on a cop

,Unnamed: 0,Name,Category,Category_1,Location,Location_1,Label,Rating,Review,avg_lunch,avg_dinner,min_lunch,max_lunch,min_dinner,max_dinner,Location_2
0,0,런던 베이글 뮤지엄 도산,"카페,디저트",카페,도산공원,강남구,웨이팅,4.4,3419,15000.0,15000.0,10000.0,20000.0,10000.0,20000.0,도산공원
2,2,라프레플루트,"카페,디저트",카페,서울숲,성동구,웨이팅,4.3,278,20000.0,20000.0,10000.0,30000.0,10000.0,30000.0,서울숲
4,4,크레이지카츠,돈가스,한식,합정,마포구,웨이팅,4.7,562,15000.0,15000.0,10000.0,20000.0,10000.0,20000.0,합정
9,9,난포,한식,한식,서울숲,성동구,웨이팅,4.3,950,30000.0,30000.0,10000.0,50000.0,10000.0,50000.0,서울숲
10,10,미뉴트빠삐용,"카페,디저트",카페,압구정로데오,강남구,웨이팅,4.0,720,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,압구정로데오
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3393,3393,온천집 스키야키,샤브샤브,한식,압구정로데오,강남구,예약-웨이팅,4.1,1133,55000.0,55000.0,20000.0,90000.0,20000.0,90000.0,압구정로데오
3394,3394,함루 본점,일식,일식,공덕,마포구,예약-웨이팅,4.4,58,60000.0,60000.0,20000.0,100000.0,20000.0,100000.0,공덕
3395,3395,이치류 한남점,양고기,육류,한남,용산구,예약-웨이팅,4.7,83,0.0,35000.0,NaN,NaN,30000.0,40000.0,한남
3398,3398,하이디라오 서초점,중식,중식,강남,강남구,예약-웨이팅,4.7,894,40000.0,40000.0,30000.0,50000.0,30000.0,50000.0,강남


In [20]:
filtered_data.avg_lunch.value_counts()
# 1- 20000, 50000, 100000, 100000+ 

avg_lunch
0.0         616
20000.0     250
15000.0     247
25000.0     194
30000.0     115
           ... 
245000.0      1
210000.0      1
190000.0      1
175000.0      1
68000.0       1
Name: count, Length: 80, dtype: int64

In [18]:
locations = input("현재 원하시는 지역구 중 1곳을 선택해주세요(1 : 강남구, 2. 마포구, 3.용산구, 4.성동구) : ")
location = int(locations)
places = ['강남구', '마포구', '용산구', '성동구']


print(location)
if location == 1:
    gangnam = places[location-1]
    print(gangnam)
    gangnam_data = filtered_data[filtered_data['Location_1'] == gangnam]
    


   
# buget_num = int(input("  "))

# print(times)
# print(buget)
gangnam_data.Location_1.value_counts()

1
a
강남구


Location_1
강남구    790
Name: count, dtype: int64

In [21]:
while True:
    try:
        lower_buget = input('현재 원하시는 최소 가격대를 적어주세요 (최소 1만원): ')
        upper_buget = input('현재 원하시는 최대 가격대를 적어주세요 (최소 30만원): ')
        if lower_buget.isdigit() and upper_buget.isdigit() and upper_buget >= lower_buget:
            print(lower_buget)
            print(upper_buget)
            break
    except:
        print("정수를 넣어주세요")
        pass

buget_l = lower_buget*10000
buget_u = upper_buget*10000
lunch_bins = [1, 20000, 30000, 50000, 100000, filtered_data['max_lunch'].max()]
dinner_bins = [1, 20000, 30000, 50000, 100000, filtered_data['max_lunch'].max()]

if buget_l >=



3
4
